In [2]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import os, sys # For filepaths


In [3]:
''' Add the datasets and libraries to the system path '''

# Find the path to our implementations
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
home_directory = os.path.dirname(parent_directory)
libraries_path = os.path.join(home_directory, 'Libraries')

# Find the path to the datasets
datasets_path = os.path.join(home_directory, 'Datasets')

# Add them both to the system path
sys.path.append(datasets_path)
sys.path.append(libraries_path)

In [4]:
''' Load the dataset '''

dataset = np.load(os.path.join(datasets_path, 'fashion_train.npy'))
print("Dataset shape: ", dataset.shape)

Dataset shape:  (10000, 785)


In [5]:
X = dataset[:, :-1]  # Define the images as all columns except the last one
y = dataset[:, -1] # Define the labels as the last column

In [ ]:
kFold=KFold(n_splits=10,shuffle=False)
for i, (train_index, test_index) in enumerate(KFold.split(X)):
    print(f"Fold {i}:")
    print(f"  Train index: {train_index}", "\n")
    print(f"  Test index: {test_index}")

